## Tutorial Notebook

This is a basic tutorial showcasing the various features of the `dual_autodiff` Python package. 

### Basic Usage

Once the package has been installed, import the dual_autodiff package:


In [1]:
# Install the dual_autodiff package
%pip install dual_autodiff

import dual_autodiff as df

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'dual_autodiff'

We can create a dual number by calling the `Dual` class and passing a real and dual part as arguments. We can then return the real and dual parts in two different ways, as shown below:

In [ ]:
# Define a dual number
x=df.Dual(2,1)

# Print the real and dual parts, either using the class attributes or designated member functions
print(x.real)
print(x.du())

We can apply basic arithmetical operations to dual numbers as we would with real numbers. This includes: addition, subtraction, multiplication, and division on either the LHS or RHS of the dual number; operate-and-assign operators; raising to a (real) exponent; comparison operators; and unary operators (i.e. inverting, or putting a '-' sign in front of a dual number).


In [ ]:
y=df.Dual(3,4)
# addition
print(x+y)

In [ ]:
# division
print(y/x)

In [ ]:
# We can also apply such operations to real numbers on both the LHS and RHS of a dual number.
print(y-5, 5-y)

In [ ]:
# The operate-and-assign operators are also defined similarly.

x*=10

print(x)

In [ ]:
# Exponentiation of dual numbers is defined for purely real exponents (i.e. the dual part must be 0).

z=df.Dual(4,9)

print(z**0.5)
print(z**df.Dual(0.5,0))

In [ ]:
# We can check for (in)equivalence of dual numbers. 

x==z

In [ ]:
# Note: '~' is the inversion operator (equivalent to raising an dual number to an exponent of (-1)).

print(~z)
print(z**(-1))

The `Dual` class also defines several common mathematical functions (see documentation for details) that can be extended to dual numbers.


In [ ]:
import numpy as np

a=df.Dual(np.pi/2, 1)

print(a.sin())

print(df.Dual(1,5).exp())

### Example - using the `Dual` class to compute derivatives

Dual numbers allow us to compute gradients to high levels of accuracy via the technique of Automatic Differentiation. 

This follows from Taylor's Theorem,
$ f(a + b \epsilon) = f(a) + f^\prime(a) (b \epsilon)$
where higher-order terms vanish since $\epsilon^2=0$. Thus the gradient at $x=a$ can be found by taking the dual part of $ f(a + \epsilon) $.

Below, we compute the gradient of $ f(x) = log(sin(x)) + x^2 cos(x) $ at $x=1.5$ using 
 - dual numbers
 - the analytical derivative
 - numerically, with decreasing step size

and compare the results.

In [ ]:
# Import the relevant packages
import numpy as np
import matplotlib.pyplot as plt

Firstly, I compute the derivative of the function using dual numbers.

In [ ]:
# Define a function that we would like to differentiate at x==1.5. 
x=df.Dual(1.5,1)
output = (x.sin()).log() + (x**2) * x.cos()

# Compute the gradient using Taylors' Theorem
grad_dual=output.dual
print(grad_dual)

Secondly, I compute the analytical derivative of the function.

In [ ]:
# Compare to the analytical result
def f_prime(x):
    return 1/np.tan(x) + 2*x*np.cos(x) - x**2 * np.sin(x)

grad_anal=f_prime(1.5)
print(grad_anal)

 This result agrees with the gradient obtained using dual numbers at the machine precision level (up to 1e-16)! This shows that the dual number method is very accurate, which is highly useful for performing automatic differentiation.

Thirdly, for comparison, I compute the numerical derivative of the function using a decreasing step size. I have also plotted the results and included the dual/analytical derivative for comparison.

In [ ]:
# Define the function f to compute the numerical gradient
def f(x):
    y = np.log(np.sin(x)) + x**2 *np.cos(x)
    return y

# Compute the numerical gradient at x==1.5    
x=1.5  

grad_num=[]
for i in range(15):
    eps=10**(-i)
    y = (f(x+eps) - f(x))/eps
    grad_num.append(y)

# Plot the gradient at each step size
step_size=[10**(-i) for i in range(15)]
plt.plot(step_size, grad_num, marker='x', label="Numerical")
plt.xlabel("Step size")
plt.xscale('log')
plt.ylabel("Gradient at $x=1.5$")
plt.title("Derivative of $ f(x) = log(sin(x)) + x^2 cos(x) $ at $x=1.5$")
plt.hlines(grad_dual, xmin=0, xmax=1, color='r', ls="-.", label= "Dual")
plt.hlines(grad_anal, xmin=0, xmax=1, color='g', ls="--", label= "Analytical")
plt.legend()
plt.show()


This looks like a good approximation to the derivative, but if we examine the errors, we find that it is not as accurate as the dual number method. I have plotted the error below.

In [ ]:
# Plot the error at each step size
step_size=[10**(-i) for i in range(15)]
plt.plot(step_size, np.abs(grad_num-grad_anal), marker='x', label="Numerical")
plt.xlabel("Step size")
plt.ylabel("Absolute value of Error")
plt.xscale('log')
plt.yscale('log')
#plt.ylim(10**(-9), 10**(0))
plt.title("|Error| in derivative of $ f(x) = log(sin(x)) + x^2 cos(x) $ at $x=1.5$")
plt.legend()
plt.show()

In summary, the numerical method is limited by the step size, which can introduce errors if it becomes too large (because the higher order terms in the Taylor expansion are not yet small enough) or too small (due to rounding error). 

## Cythonized Package

In [ ]:
import dual_autodiff_x as dfx

In [ ]:
x=dfx.Dual(1.5,1)
x**50

In [ ]:
%%timeit
# Define a function that we would like to differentiate. 
output = (x.sin()).log() + (x**2) * x.cos()

# Compute the gradient at x==1.5 using Taylors' Theorem
x=dfx.Dual(1.5,1)
grad_dual=output.dual
print(grad_dual)